In [44]:
import pygame as pg
from pygame.locals import *
import sys
import time
import math
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#initialize global variables
cell_num = 0
checked = None
num_selected = None
draw = False
width = 400
height = 400
white = (255, 255, 255)
line_color = (10,10,10)
#TicTacToe 3x3 board
board = [[None]*9,[None]*9,[None]*9, [None]*9,[None]*9,[None]*9,
      [None]*9,[None]*9,[None]*9]


pg.init()
fps = 30
CLOCK = pg.time.Clock()
screen = pg.display.set_mode((width, height), pg.RESIZABLE)
pg.display.set_caption('Sudoku')

#loading the images
opening = pg.image.load('tic tac opening.png')
x_img = pg.image.load('x.png')
o_img = pg.image.load('o.png')
#resizing images
x_img = pg.transform.scale(x_img, (40,40))
o_img = pg.transform.scale(o_img, (80,80))
opening = pg.transform.scale(opening, (width, height))

    
def draw_status():
    global draw
        
    #font = pg.font.Font(None, 30)
    #text = font.render(message, 1, (255, 255, 255))
    
    # copy the rendered message onto the board
    #screen.fill ((0, 0, 0), (0, 400, 500, 100))
    #text_rect = text.get_rect(center=(width/2, 500-50))
    #screen.blit(text, text_rect)
    #pg.display.update()
    
def check_win():
    global board, winner, draw
    
    draw_status()
    
def draw_num(row, col, board_dim):
    global board, cell_num
    
    posx = board_dim/9*(col-1)+board_dim/25
    posy = board_dim/9*(row-1)+board_dim/30
 
    board[row-1][col-1] = cell_num
    
    font = pg.font.Font(None, int(board_dim/10))
    text = font.render(str(1), 1, (0, 0, 0))
    screen.blit(text, (posx, posy))
    
    pg.display.update()
    
def userClick():
    #get coordinates of mouse click
    global checked, num_selected
    
    x, y = pg.mouse.get_pos()
    width = screen.get_width()
    height = screen.get_height()
    dim = int(min(screen.get_width(), screen.get_height()))
    board_dim = dim

    if width > height:
        if width > height*1.5:
            board_dim = dim
        else:
            board_dim = 2*width/3
    elif width <= height:
        if width*1.5 < height:
            board_dim = dim
        else:
            board_dim = 2*height/3

    col = math.ceil(x/(board_dim/9))
    row = math.ceil(y/(board_dim/9))
    
    if(row>0 and col>0 and row<10 and col<10 and board[row-1][col-1]==None):
        if checked==(row-1, col-1):
            checked = None
            posx1 = int(board_dim*(col-1)/9)+board_dim/200
            posy1 = int(board_dim*(row-1)/9)+board_dim/200
            posx2 = int(board_dim*col/9)-board_dim/200
            posy2 = int(board_dim*row/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(1), 1, (0, 0, 0))

            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
            return
        else:
            if checked!=None:
                posx1 = int(board_dim*(checked[1])/9)+board_dim/200
                posy1 = int(board_dim*(checked[0])/9)+board_dim/200
                posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
                posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200

                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(1), 1, (0, 0, 0))

                screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
                
            checked = (row-1, col-1)

            posx1 = int(board_dim*(col-1)/9)+board_dim/200
            posy1 = int(board_dim*(row-1)/9)+board_dim/200
            posx2 = int(board_dim*col/9)-board_dim/200
            posy2 = int(board_dim*row/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(1), 1, (0, 0, 0))

            screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
            return

    found_num = False
    if checked != None:
        if height >= width:
            if(y>(board_dim+(height-board_dim)/2) and x<=board_dim):
                num_selected = math.ceil(x/(board_dim/9))
                found_num = True
        else:
            if(x>(board_dim+(width-board_dim)/2) and y<=board_dim):
                num_selected = math.ceil(y/(board_dim/9))
                found_num = True
                
        if found_num:
            posx1 = int(board_dim*(checked[1])/9)+board_dim/200
            posy1 = int(board_dim*(checked[0])/9)+board_dim/200
            posx2 = int(board_dim*(checked[1]+1)/9)-board_dim/200
            posy2 = int(board_dim*(checked[0]+1)/9)-board_dim/200

            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(1), 1, (0, 0, 0))
            screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

            posx = board_dim/9*checked[1]+board_dim/25
            posy = board_dim/9*checked[0]+board_dim/30

            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(num_selected), 1, (0, 0, 0))
            screen.blit(text, (posx, posy))

            board[checked[0]][checked[1]] = num_selected
            checked = None
            print(num_selected)

        
def CreateWindow(width=400, height=600):
    '''Updates the window width and height '''
    global checked
    
    
    screen = pg.display.set_mode((width, height), RESIZABLE)
    pg.display.update()
    screen.fill(white)
    dim = int(min(width, height))
    board_dim = dim
    
    if width > height:
        if width > height*1.5:
            board_dim = dim
        else:
            board_dim = 2*width/3
    elif width <= height:
        if width*1.5 < height:
            board_dim = dim
        else:
            board_dim = 2*height/3
    
    # Drawing vertical lines
    for num in range(10):
        line_thick = 1

        if num%3==0:
            line_thick = 3
        
        # Vertical lines of board
        pg.draw.line(screen, line_color, (board_dim*num/9.0, 0), (board_dim*num/9.0, board_dim), line_thick)
        # Horizontal Lines of board
        pg.draw.line(screen, line_color, (0, board_dim*num/9.0), (board_dim, board_dim*num/9.0), line_thick)
        
        if height>=width:  # Game buttons on bottom if height greater than or equal width
            pg.draw.line(screen, line_color, (board_dim*num/9.0, board_dim+(height-board_dim)/2), (board_dim*num/9.0, height), 1)
        else:  # Game buttons on side if width greater height
            pg.draw.line(screen, line_color, (board_dim+(width-board_dim)/2, board_dim*num/9.0), (width, board_dim*num/9.0), 1)

        for n in range(10):
            if board[num-1][n-1]!=None:
                posx = board_dim/9*(n-1)+board_dim/25
                posy = board_dim/9*(num-1)+board_dim/30
                
                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(board[num-1][n-1]), 1, (0, 0, 0))
                screen.blit(text, (posx, posy))
                
            if checked==(num-1, n-1):
                posx1 = int(board_dim*(n-1)/9)+board_dim/200
                posy1 = int(board_dim*(num-1)/9)+board_dim/200
                posx2 = int(board_dim*n/9)-board_dim/200
                posy2 = int(board_dim*num/9)-board_dim/200
                
                font = pg.font.Font(None, int(board_dim/10))
                text = font.render(str(1), 1, (0, 0, 0))
                #screen.blit(text, (posx, posy))

                screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
    
    
    if height>=width:
        pg.draw.line(screen, line_color, (0, board_dim+(height-board_dim)/2), (board_dim, board_dim+(height-board_dim)/2), 1)
        pg.draw.line(screen, line_color, (0, height), (board_dim, height), 1)
        
        for num in range(1, 10):
            posx = board_dim/9*(num-1)+board_dim/25
            posy = board_dim+3*(height-board_dim)/4
            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(num), 1, (0, 0, 0))
            screen.blit(text, (posx, posy))
    else:
        pg.draw.line(screen, line_color, (board_dim+(width-board_dim)/2, 0), (board_dim+(width-board_dim)/2, board_dim), 1)
        pg.draw.line(screen, line_color, (width, 0), (width, board_dim), 1)
        
        for num in range(1, 10):
            posx = board_dim+3*(width-board_dim)/4
            posy = board_dim/9*(num-1)+board_dim/25
            font = pg.font.Font(None, int(board_dim/10))
            text = font.render(str(num), 1, (0, 0, 0))
            screen.blit(text, (posx, posy))
        
def reset_game():
    global board, winner, cell_num, draw
    
    time.sleep(3)
    cell_num = 0
    draw = False
    game_opening()
    winner=None
    board = [[None]*3,[None]*3,[None]*3]
    

CreateWindow()

# run the game loop forever
while(True):
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == VIDEORESIZE:
            CreateWindow(event.w, event.h)
        elif event.type is MOUSEBUTTONDOWN:
            # the user clicked; place an X or O
            userClick()
            #if(winner or draw):
                #reset_game()
    pg.display.update()
    CLOCK.tick(fps)

1
2
9
7
1
6
8


SystemExit: 

In [221]:
import copy
import random

square_length = 3
side_length = square_length*square_length
current_sub_square = 0
board= [0][0]
sub_squares = {}
rows = {}
columns = {}
temp_sub_squares = {}
temp_columns = {}
num_found = False
restart = False
iterations = 0
entries = 0


def gameSetup():
    global entries, iterations, restart, sub_squares, rows, columns, temp_sub_squares 
    global temp_columns, board
    
    gameSetupRestart()
    
    while(entries<side_length*side_length):
        for row_num in range(side_length):
            col_num = 0
            while(col_num<side_length):
                current_sub_square = math.floor(row_num/square_length)*square_length + math.floor(col_num/square_length)
                num_found =False
                
                while(not num_found):
                    temp = []
                    for num in rows[row_num]:
                        if num not in temp and num in columns[col_num] and num in sub_squares[current_sub_square]:
                            temp.append(num)
                            
                    if(len(temp)==0):
                        sub_squares = subSquareReset(sub_squares, current_sub_square, row_num)
                        rows = rowReset(rows, row_num)
                        columns = columnReset(columns, col_num)
                        boardRowReset(board, row_num)
                        col_num = 0
                        current_sub_square = math.floor(row_num/square_length)*square_length + math.floor(col_num/square_length)
                        iterations+=1
                        
                        if(iterations>side_length*side_length):
                            restart = True
                            break
                    else:
                        seed = random.randint(0, len(temp)-1)
                        value = temp[seed]
                        board[row_num][col_num] = value
                        temp_columns[col_num].append(value)
                        temp_sub_squares[current_sub_square][math.floor(row_num%square_length)].append(value)
                        sub_squares[current_sub_square].remove(value)
                        rows[row_num].remove(value)
                        columns[col_num].remove(value)
                        entries+=1
                        col_num+=1
                        num_found = True
                
                if(restart):
                    break
                    
            for m in range(side_length):
                temp_columns[m] = []
            
            if(restart):
                gameSetupRestart()
                break
    
    if(not boardChecker(board)):
        gameSetup()
    else:
        for i in range(side_length):
            for j in range(side_length):
                board[i][j]+=1
                        
                    

def gameSetupRestart():
    global entries, iterations, restart, sub_squares, rows, columns 
    global temp_sub_squares, temp_columns, board
    
    entries = 0
    iterations = 0
    restart = False
    board = [[-1 for i in range(side_length)] for i in range(side_length)]
    
    for i in range(side_length):
        sub_squares[i] = []
        rows[i] = []
        columns[i] = []
        temp_sub_squares[i] = {}
        temp_columns[i] = []
        
    for j in range(side_length):
        for k in range(side_length):
            sub_squares[j].append(k)
            rows[j].append(k)
            columns[j].append(k)
            
        for m in range(side_length):
            for n in range(side_length):
                temp_sub_squares[m][n] = []


def subSquareReset(m, sub_square, row_num):
    num_to_clear = sub_square%square_length
    sub_row = row_num%square_length

    for i in range(num_to_clear+1):
        temp = copy.deepcopy(temp_sub_squares[sub_square-i][sub_row])
        m[sub_square-i].extend(temp)
        temp_sub_squares[sub_square-i][sub_row] = []
 
    return m

def rowReset(m, row_num):
    m[row_num] = []
    
    for i in range(side_length):
        m[row_num].append(i)
    return m

def columnReset(m, col_num):
    for i in range(col_num):
        for num in temp_columns[i]:
            m[i].append(num)
            
        temp_columns[i] = []
        
    return m

def boardRowReset(board, row_num):
    for k in range(side_length):
        board[row_num][k] = -1

def boardChecker(board):
    for i in range(side_length):
        for j in range(side_length):
            for k in range(j+1, side_length):
                if board[i][j]==board[i][k] or board[j][i]==board[k][i]:
                    return False
    entries = 0
    row = 0
    col = 0
    modifier = 0
    values = {}
    
    for p in range(side_length):
        values[p] = []
    
    while(entries<side_length*side_length):
        for i in range(row, row+square_length):
            for j in range(col, col+square_length):
                entries+=1
                
                if len(values[board[i][j]])==0:
                    values[board[i][j]].append(1)
                else:
                    return False
        
        for m in range(side_length):
            values[m] = []
        
        modifier+=1
        
        if modifier%square_length==0:
            col = 0
            row+= square_length
        else:
            col+=square_length
    
    return True

def printBoard():
    for i in range(side_length):
        for j in range(side_length):
            print(board[i][j], end=" ")
        print(' ')
    print(' ')

In [225]:
gameSetup()


[[9, 6, 3, 1, 4, 7, 8, 5, 2], [5, 4, 8, 9, 6, 2, 7, 3, 1], [1, 7, 2, 8, 5, 3, 6, 9, 4], [8, 1, 4, 2, 9, 5, 3, 7, 6], [6, 2, 9, 3, 7, 8, 1, 4, 5], [3, 5, 7, 4, 1, 6, 2, 8, 9], [4, 3, 1, 7, 2, 9, 5, 6, 8], [7, 9, 5, 6, 8, 1, 4, 2, 3], [2, 8, 6, 5, 3, 4, 9, 1, 7]]
[[7, 3, 2, 6, 9, 8, 5, 4, 1], [5, 9, 4, 1, 2, 3, 6, 7, 8], [6, 8, 1, 4, 7, 5, 3, 9, 2], [2, 5, 8, 7, 4, 6, 9, 1, 3], [3, 6, 9, 8, 1, 2, 7, 5, 4], [4, 1, 7, 5, 3, 9, 8, 2, 6], [8, 7, 6, 2, 5, 4, 1, 3, 9], [9, 4, 5, 3, 8, 1, 2, 6, 7], [1, 2, 3, 9, 6, 7, 4, 8, 5]]
[[1, 8, 7, 6, 4, 5, 2, 3, 9], [5, 2, 6, 9, 7, 3, 4, 8, 1], [3, 4, 9, 2, 1, 8, 6, 7, 5], [2, 5, 8, 1, 9, 6, 3, 4, 7], [7, 1, 3, 8, 5, 4, 9, 2, 6], [9, 6, 4, 3, 2, 7, 1, 5, 8], [6, 7, 2, 5, 3, 1, 8, 9, 4], [8, 9, 5, 4, 6, 2, 7, 1, 3], [4, 3, 1, 7, 8, 9, 5, 6, 2]]
[[3, 9, 6, 1, 7, 4, 2, 8, 5], [7, 5, 4, 2, 8, 3, 9, 1, 6], [8, 1, 2, 9, 5, 6, 7, 3, 4], [1, 7, 5, 3, 6, 8, 4, 2, 9], [9, 6, 3, 4, 2, 5, 8, 7, 1], [4, 2, 8, 7, 1, 9, 5, 6, 3], [2, 4, 1, 6, 9, 7, 3, 5, 8], [5, 3, 7, 

[[3, 5, 6, 1, 4, 9, 2, 7, 8], [8, 7, 4, 2, 5, 6, 3, 9, 1], [2, 9, 1, 7, 8, 3, 4, 6, 5], [5, 3, 9, 4, 7, 8, 6, 1, 2], [6, 8, 2, 5, 9, 1, 7, 3, 4], [4, 1, 7, 6, 3, 2, 8, 5, 9], [9, 6, 5, 8, 2, 7, 1, 4, 3], [1, 2, 3, 9, 6, 4, 5, 8, 7], [7, 4, 8, 3, 1, 5, 9, 2, 6]]
[[7, 2, 8, 1, 6, 4, 3, 9, 5], [3, 9, 1, 2, 5, 8, 6, 7, 4], [4, 5, 6, 7, 9, 3, 1, 2, 8], [8, 7, 9, 4, 1, 5, 2, 3, 6], [2, 4, 5, 3, 7, 6, 8, 1, 9], [6, 1, 3, 8, 2, 9, 5, 4, 7], [1, 6, 2, 5, 4, 7, 9, 8, 3], [9, 8, 7, 6, 3, 1, 4, 5, 2], [5, 3, 4, 9, 8, 2, 7, 6, 1]]
[[6, 1, 2, 7, 9, 3, 8, 5, 4], [5, 7, 4, 8, 1, 2, 9, 3, 6], [9, 3, 8, 5, 4, 6, 1, 7, 2], [1, 6, 7, 2, 3, 8, 4, 9, 5], [4, 9, 5, 6, 7, 1, 2, 8, 3], [8, 2, 3, 4, 5, 9, 7, 6, 1], [7, 8, 6, 1, 2, 5, 3, 4, 9], [3, 4, 1, 9, 6, 7, 5, 2, 8], [2, 5, 9, 3, 8, 4, 6, 1, 7]]
[[1, 5, 9, 3, 2, 6, 4, 8, 7], [8, 4, 6, 1, 5, 7, 3, 2, 9], [2, 3, 7, 8, 4, 9, 6, 5, 1], [4, 1, 3, 5, 6, 8, 7, 9, 2], [6, 7, 2, 9, 3, 1, 5, 4, 8], [9, 8, 5, 4, 7, 2, 1, 6, 3], [5, 2, 4, 7, 8, 3, 9, 1, 6], [7, 9, 8, 

[[1, 7, 4, 5, 2, 9, 6, 8, 3], [6, 2, 8, 3, 7, 1, 5, 9, 4], [3, 5, 9, 8, 6, 4, 2, 1, 7], [7, 1, 5, 4, 9, 3, 8, 2, 6], [4, 8, 6, 2, 5, 7, 1, 3, 9], [9, 3, 2, 6, 1, 8, 4, 7, 5], [8, 4, 7, 1, 3, 6, 9, 5, 2], [5, 6, 3, 9, 8, 2, 7, 4, 1], [2, 9, 1, 7, 4, 5, 3, 6, 8]]
[[2, 5, 1, 4, 3, 7, 8, 6, 9], [7, 4, 9, 8, 1, 6, 3, 5, 2], [6, 8, 3, 2, 9, 5, 4, 7, 1], [1, 3, 6, 9, 4, 2, 5, 8, 7], [5, 2, 8, 7, 6, 1, 9, 4, 3], [4, 9, 7, 5, 8, 3, 1, 2, 6], [3, 1, 5, 6, 2, 4, 7, 9, 8], [8, 6, 4, 3, 7, 9, 2, 1, 5], [9, 7, 2, 1, 5, 8, 6, 3, 4]]
[[6, 1, 7, 3, 9, 2, 4, 8, 5], [2, 3, 9, 4, 5, 8, 1, 6, 7], [5, 8, 4, 1, 7, 6, 9, 3, 2], [8, 5, 2, 7, 4, 9, 6, 1, 3], [7, 9, 1, 5, 6, 3, 2, 4, 8], [3, 4, 6, 8, 2, 1, 5, 7, 9], [4, 6, 5, 2, 8, 7, 3, 9, 1], [1, 2, 8, 9, 3, 4, 7, 5, 6], [9, 7, 3, 6, 1, 5, 8, 2, 4]]
[[6, 2, 3, 4, 8, 5, 1, 9, 7], [4, 9, 1, 3, 2, 7, 5, 8, 6], [8, 5, 7, 1, 6, 9, 3, 4, 2], [7, 6, 9, 2, 5, 4, 8, 3, 1], [1, 8, 4, 7, 3, 6, 9, 2, 5], [5, 3, 2, 9, 1, 8, 6, 7, 4], [9, 7, 8, 6, 4, 1, 2, 5, 3], [2, 1, 5, 

[[2, 6, 1, 7, 9, 5, 4, 8, 3], [8, 7, 3, 1, 6, 4, 9, 2, 5], [4, 9, 5, 2, 8, 3, 1, 6, 7], [3, 2, 9, 5, 4, 8, 7, 1, 6], [6, 8, 7, 3, 1, 9, 5, 4, 2], [1, 5, 4, 6, 7, 2, 8, 3, 9], [5, 3, 8, 4, 2, 7, 6, 9, 1], [7, 4, 6, 9, 3, 1, 2, 5, 8], [9, 1, 2, 8, 5, 6, 3, 7, 4]]
[[5, 7, 1, 8, 3, 2, 9, 6, 4], [6, 8, 9, 5, 4, 7, 3, 2, 1], [4, 3, 2, 6, 9, 1, 7, 5, 8], [3, 2, 7, 1, 5, 8, 4, 9, 6], [9, 6, 5, 2, 7, 4, 8, 1, 3], [8, 1, 4, 9, 6, 3, 5, 7, 2], [7, 4, 6, 3, 1, 9, 2, 8, 5], [2, 5, 3, 7, 8, 6, 1, 4, 9], [1, 9, 8, 4, 2, 5, 6, 3, 7]]
[[1, 4, 7, 2, 6, 8, 3, 9, 5], [3, 9, 8, 1, 5, 4, 2, 7, 6], [5, 2, 6, 7, 9, 3, 8, 4, 1], [2, 5, 3, 6, 8, 7, 9, 1, 4], [8, 6, 4, 9, 1, 5, 7, 3, 2], [9, 7, 1, 4, 3, 2, 5, 6, 8], [6, 3, 9, 5, 2, 1, 4, 8, 7], [7, 1, 5, 8, 4, 9, 6, 2, 3], [4, 8, 2, 3, 7, 6, 1, 5, 9]]
[[4, 2, 7, 3, 1, 9, 5, 8, 6], [6, 3, 1, 2, 5, 8, 9, 7, 4], [5, 9, 8, 6, 7, 4, 3, 2, 1], [8, 1, 6, 4, 2, 3, 7, 9, 5], [9, 5, 3, 1, 6, 7, 8, 4, 2], [2, 7, 4, 9, 8, 5, 6, 1, 3], [7, 4, 9, 5, 3, 1, 2, 6, 8], [3, 8, 2, 

In [84]:
a = [[1,2], [1,2]]
a[0][0]

1

In [224]:
board

[[2, 9, 6, 1, 5, 4, 8, 3, 7],
 [7, 1, 5, 3, 9, 8, 2, 4, 6],
 [4, 8, 3, 2, 6, 7, 9, 5, 1],
 [8, 3, 7, 5, 4, 2, 1, 6, 9],
 [5, 2, 9, 8, 1, 6, 3, 7, 4],
 [6, 4, 1, 7, 3, 9, 5, 2, 8],
 [3, 7, 4, 9, 2, 1, 6, 8, 5],
 [1, 5, 8, 6, 7, 3, 4, 9, 2],
 [9, 6, 2, 4, 8, 5, 7, 1, 3]]